In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# DOA 7

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=(3,3), stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=(3,3), stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = F.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=181):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=(4,3), stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=1)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=1)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=1)
        self.linear = nn.Linear(25, num_classes)
        #self.flat = nn.Conv2d(in_channels=512, out_channels=1, kernel_size=1, stride=1)
        self.flat = nn.Sequential(nn.Conv2d(in_channels=512, out_channels=1, kernel_size=1, stride=1))

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        # print("Input ==> ", x.size())
        out = F.relu(self.bn1(self.conv1(x)))
        # print("F.relu(self.bn1(self.conv1(x))) ==> ", x.size())
        out = self.layer1(out)
        # print("layer1 ==> ", x.size())
        out = self.layer2(out)
        # print("layer2 ==> ", x.size())
        out = self.layer3(out)
        # print("layer3 ==> ", x.size())
        out = self.layer4(out)
        # print("layer4 ==>", out.size())
        out = F.avg_pool2d(out, (1,4))
        # print("avg_pool2d ===>", out.size())
        # out = out.view(out.size(0), -1)
        # print("out.view ===>", out.size())
        out = self.linear(out)
        out = self.flat(out)
        # print("Out ===>", out.size())
        return out


def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])


def ResNet34():
    return ResNet(BasicBlock, [3, 4, 6, 3])

def ResNet101():
    return ResNet(BasicBlock, [3, 4, 23, 3])


if __name__ == "__main__":
    image = torch.rand(1, 3, 8, 100)
    model = ResNet34()
    model(image)


In [2]:
import scipy.io as sio
import numpy as np
import torch
from torch import nn, optim
import torchvision
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import numpy as np 
import math
import pandas as pd
import cmath
import scipy.io as sio
import numpy as np
import torch
from torch import nn, optim
import torchvision
from torch.utils.data import Dataset, DataLoader
import numpy as np 
import math
import pandas as pd
import cmath
import os
from os import listdir
from os.path import isfile, join
#from unet import UNet
# from auto import encoder, decoder

from collections import OrderedDict
from torch.utils.data.sampler import SubsetRandomSampler
from torch.autograd import Variable
#==========================================================================
# For Plotting loss graph
# Bokeh
from bokeh.io import curdoc
from bokeh.layouts import column
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure

from functools import partial
from threading import Thread
from tornado import gen
# from AttRCNN_UNet import Att_R2U
import logging
logging.basicConfig(format='%(asctime)s - %(message)s', level=logging.INFO)
import sys
# from dataloader import norm

# import wandb
# import logging
# logging.basicConfig(format='%(asctime)s - %(message)s', level=logging.INFO)

In [ ]:
train_dataset_list = []
test_dataset_list = []

In [ ]:
import csv
from sklearn.model_selection import train_test_split
from sys import getsizeof

In [ ]:
df1  = sio.loadmat("/content/drive/MyDrive/DOA/DOA7/SNR_NS_0_7_100000.mat")
df2  = sio.loadmat("/content/drive/MyDrive/DOA/DOA7/SNR_NS_10_7_100000.mat")
df3  = sio.loadmat("/content/drive/MyDrive/DOA/DOA7/SNR_NS_20_7_100000.mat")
df4  = sio.loadmat("/content/drive/MyDrive/DOA/DOA7/SNR_NS_30_7_100000.mat")
df5  = sio.loadmat("/content/drive/MyDrive/DOA/DOA7/SNR_NS_40_7_100000.mat")

In [ ]:
df = [df1, df2, df3, df4, df5]
# a = getsizeof(df)

In [ ]:
new_train_dataset_list = []
new_test_dataset_list = []
train_dataset_list_label = []
test_dataset_list_label = []

In [ ]:
def create_dataset(df):
    data = np.transpose(df['NS_data'], (2, 0, 1))
    label = df['DOA']
    X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.15, random_state=42)
    new_train_dataset_list.extend(X_train)
    new_test_dataset_list.append(X_test.tolist())
    train_dataset_list_label.extend(y_train)
    test_dataset_list_label.append(y_test.tolist())

In [ ]:
for file in df:
    create_dataset(file)

In [ ]:
del df
del df1
del df2
del df3
del df4
del df5

In [ ]:
len(test_dataset_list_label[0])

15000

In [ ]:
for idx, data in enumerate(new_train_dataset_list):
    new = np.zeros((3, 8, 100))
    for j in range(0, data.shape[0]):
        for k in range(0, data.shape[1]):
            new[0][j][k] = data[j][k].real
            new[1][j][k] = data[j][k].imag
            new[2][j][k] = cmath.phase(data[j][k])
    new_train_dataset_list[idx] = new

In [ ]:
for idx, data in enumerate(new_test_dataset_list):
    for i, ndata in enumerate(data):
        new = np.zeros((3, 8, 100))
        for j in range(0, 8):
            for k in range(0, 100):
                new[0][j][k] = ndata[j][k].real
                new[1][j][k] = ndata[j][k].imag
                new[2][j][k] = cmath.phase(ndata[j][k])
        new_test_dataset_list[idx][i] = new

In [ ]:
def get_data(train, test, batch_size, train_bool=True):
    class DOA_dataset(Dataset):
        def __init__(self, train, test):
            self.x = torch.from_numpy(np.array(train))
            self.y = torch.from_numpy(np.asarray(test))
            self.n_sample = len(self.y)
        def __getitem__(self, index):
            return self.x[index], self.y[index]
        def __len__(self):
            return self.n_sample


    dataset = DOA_dataset(train, test)
    

    loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=train_bool)
    return loader


In [ ]:
doa2_train_loader = get_data(new_train_dataset_list, train_dataset_list_label, 64, True)

In [ ]:
del new_train_dataset_list
del train_dataset_list_label

In [ ]:
test_dataset_list = []

In [ ]:
for idx, data in enumerate(new_test_dataset_list):
    print(type(data))
    f = get_data(data, test_dataset_list_label[idx], 128, False)
    test_dataset_list.append(f)

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>


In [ ]:
# len(test_dataset_list[0])*128

In [ ]:
del new_test_dataset_list
del new

In [ ]:
num_epochs = 50
doa = 7
weights_dir = "/content/drive/MyDrive/DOA/doa_weights/"

autoencoder = ResNet34()
criterion = nn.CrossEntropyLoss()
if ('DOA_{}_model.pth'.format(doa) in [f for f in listdir(weights_dir) if isfile(join(weights_dir, f))]):
    print("Pre-trained available for DOA_{}_model.pth".format(doa))
    autoencoder = torch.load(os.path.join(weights_dir, 'DOA_{}_model.pth'.format(doa)))
if torch.cuda.is_available():
	print(torch.cuda.get_device_name(0))
	classification_model = autoencoder.cuda()
	optimizer = optim.AdamW(classification_model.parameters(), lr=0.001, weight_decay=1e-5)
	criterion = criterion.cuda()

Pre-trained available for DOA_7_model.pth
Tesla P100-PCIE-16GB


In [ ]:
acc_res = {
    "training": [],
    0: [],
    10: [],
    20: [],
    30: [],
    40: []
}
loss_res = {
    "training": [],
    0: [],
    10: [],
    20: [],
    30: [],
    40: []
}

In [ ]:
doa

7

In [ ]:
def train():
    print("Training Starts !!!!!!!")
    
    best_valid_loss = float('Inf')
    for i in range(num_epochs):
        training_loss = 0
        train_correct = 0
        train_total = 0
        epoch_loss = 0.0
        training_mae = 0.0
        classification_model.train()
        for j,(features, labels) in enumerate(doa2_train_loader, 0):
            features, labels = Variable(features.cuda()), Variable(labels.cuda())
            optimizer.zero_grad()
            enn = classification_model(features.float())
            auto_outputs = torch.transpose(enn, 2, 3)
            auto_outputs = torch.reshape(auto_outputs.cuda(), (auto_outputs.shape[0], 181, 7))
            losss = criterion(auto_outputs.cuda(), labels.type(torch.LongTensor).cuda())
            losss.backward()
            optimizer.step()
#           exp_scheduler.step()
            training_loss += losss.item()

            _, pred = torch.max(auto_outputs, 1)

            train_total+= labels.reshape(-1).size(0)

            train_correct+=(pred.reshape(-1).cuda() == labels.reshape(-1)).sum().item()

            epoch_loss += auto_outputs.shape[0] * losss.item()
            training_mae += torch.abs(pred.reshape(-1).cuda() - labels.reshape(-1)).sum().item()  

        loss_res['training'].append(training_loss/len(doa2_train_loader))
        acc_res['training'].append((100*(train_correct/train_total)))

        print('Epoch [{}/{}], Training Loss: {:.4f}, Training Accuracy: {:.4f}'
                      .format(i+1, num_epochs, training_loss/len(doa2_train_loader), (100*(train_correct/train_total))))
        
        # Validation for each SNR value
        classification_model.eval()
        total_valdation_loss = 0
        for val_data in range(0, len(test_dataset_list)):
            
            validation_loss = 0
            validation_acc = 0
            validation_mae = 0.0
            val_correct = 0
            val_total = 0

            with torch.no_grad():
                for features, labels in test_dataset_list[val_data]:
                    features, labels = Variable(features.cuda()), Variable(labels.cuda())
                    enn = classification_model(features.float())
                    auto_outputs = torch.transpose(enn, 2, 3)
                    auto_outputs = torch.reshape(auto_outputs, (auto_outputs.shape[0], 181, 7))
                    loss = criterion(auto_outputs.cuda(), labels.type(torch.LongTensor).cuda())

                    _, pred = torch.max(auto_outputs, 1)
                    val_total+= labels.reshape(-1).size(0)
                    val_correct+=(pred.reshape(-1).cuda() == labels.reshape(-1)).sum().item()
                    validation_loss += loss.item()
                    validation_mae += torch.abs(pred.reshape(-1).cuda() - labels.reshape(-1)).sum().item()
                # print(val_correct, val_total)
                loss_res[10*val_data].append(validation_loss/len(test_dataset_list[val_data]))
                acc_res[10*val_data].append((100*(val_correct/val_total)))

                # print(val_data*10, "dB SNR is validated")
                print('SNR [{}d], Validation Loss: {:.4f}, Validation Accuracy: {:.4f}'
                          .format(val_data*10, validation_loss/len(test_dataset_list[val_data]), (100*(val_correct/val_total))))

            total_valdation_loss+=validation_loss

            if best_valid_loss > total_valdation_loss:
                best_valid_loss = total_valdation_loss 
                # Saving Best Pre-Trained Model as .pth file
                torch.save( model, "/content/drive/MyDrive/DOA/doa_weights/DOA_{}_model.pth".format(doa))
        if i%10 == 0:
          ddf = pd.DataFrame(acc_res)
          ddf.to_csv("/content/drive/MyDrive/DOA/doa_weights/res_DOA_{}_model.csv".format(doa))
        print("\n")  

train()
print("Training Complete")

Training Starts !!!!!!!
Epoch [1/50], Training Loss: 4.6507, Training Accuracy: 2.1731
SNR [0d], Validation Loss: 4.4668, Validation Accuracy: 2.9971
SNR [10d], Validation Loss: 4.0201, Validation Accuracy: 4.7829
SNR [20d], Validation Loss: 3.9130, Validation Accuracy: 5.4295
SNR [30d], Validation Loss: 3.9065, Validation Accuracy: 5.2695
SNR [40d], Validation Loss: 3.9117, Validation Accuracy: 5.3676


Epoch [2/50], Training Loss: 3.5378, Training Accuracy: 7.8957
SNR [0d], Validation Loss: 3.8061, Validation Accuracy: 6.5276
SNR [10d], Validation Loss: 3.1746, Validation Accuracy: 10.7057
SNR [20d], Validation Loss: 3.0134, Validation Accuracy: 11.8171
SNR [30d], Validation Loss: 3.0086, Validation Accuracy: 11.7000
SNR [40d], Validation Loss: 3.0050, Validation Accuracy: 11.6152


Epoch [3/50], Training Loss: 2.9469, Training Accuracy: 13.4700
SNR [0d], Validation Loss: 3.4849, Validation Accuracy: 9.1467
SNR [10d], Validation Loss: 2.7766, Validation Accuracy: 15.2581
SNR [20d], V

In [ ]:
def train():
    print("Training Starts !!!!!!!")
    
    best_valid_loss = float('Inf')
    for i in range(num_epochs):
        training_loss = 0
        train_correct = 0
        train_total = 0
        epoch_loss = 0.0
        training_mae = 0.0
        classification_model.train()
        for j,(features, labels) in enumerate(doa2_train_loader, 0):
            features, labels = Variable(features.cuda()), Variable(labels.cuda())
            optimizer.zero_grad()
            enn = classification_model(features.float())
            auto_outputs = torch.transpose(enn, 2, 3)
            auto_outputs = torch.reshape(auto_outputs.cuda(), (auto_outputs.shape[0], 181, 7))
            losss = criterion(auto_outputs.cuda(), labels.type(torch.LongTensor).cuda())
            losss.backward()
            optimizer.step()
#           exp_scheduler.step()
            training_loss += losss.item()

            _, pred = torch.max(auto_outputs, 1)

            train_total+= labels.reshape(-1).size(0)

            train_correct+=(pred.reshape(-1).cuda() == labels.reshape(-1)).sum().item()

            epoch_loss += auto_outputs.shape[0] * losss.item()
            training_mae += torch.abs(pred.reshape(-1).cuda() - labels.reshape(-1)).sum().item()  

        loss_res['training'].append(training_loss/len(doa2_train_loader))
        acc_res['training'].append((100*(train_correct/train_total)))

        print('Epoch [{}/{}], Training Loss: {:.4f}, Training Accuracy: {:.4f}'
                      .format(i+1, num_epochs, training_loss/len(doa2_train_loader), (100*(train_correct/train_total))))
        
        # Validation for each SNR value
        classification_model.eval()
        total_valdation_loss = 0
        for val_data in range(0, len(test_dataset_list)):
            
            validation_loss = 0
            validation_acc = 0
            validation_mae = 0.0
            val_correct = 0
            val_total = 0

            with torch.no_grad():
                for features, labels in test_dataset_list[val_data]:
                    features, labels = Variable(features.cuda()), Variable(labels.cuda())
                    enn = classification_model(features.float())
                    auto_outputs = torch.transpose(enn, 2, 3)
                    auto_outputs = torch.reshape(auto_outputs, (auto_outputs.shape[0], 181, 7))
                    loss = criterion(auto_outputs.cuda(), labels.type(torch.LongTensor).cuda())

                    _, pred = torch.max(auto_outputs, 1)
                    val_total+= labels.reshape(-1).size(0)
                    val_correct+=(pred.reshape(-1).cuda() == labels.reshape(-1)).sum().item()
                    validation_loss += loss.item()
                    validation_mae += torch.abs(pred.reshape(-1).cuda() - labels.reshape(-1)).sum().item()
                # print(val_correct, val_total)
                loss_res[10*val_data].append(validation_loss/len(test_dataset_list[val_data]))
                acc_res[10*val_data].append((100*(val_correct/val_total)))

                # print(val_data*10, "dB SNR is validated")
                print('SNR [{}d], Validation Loss: {:.4f}, Validation Accuracy: {:.4f}'
                          .format(val_data*10, validation_loss/len(test_dataset_list[val_data]), (100*(val_correct/val_total))))

            total_valdation_loss+=validation_loss

            if best_valid_loss > total_valdation_loss:
                best_valid_loss = total_valdation_loss 
                # Saving Best Pre-Trained Model as .pth file
                torch.save( model, "/content/drive/MyDrive/DOA/doa_weights/DOA_{}_model.pth".format(doa))
        if i%10 == 0:
          ddf = pd.DataFrame(acc_res)
          ddf.to_csv("/content/drive/MyDrive/DOA/doa_weights/res_DOA_{}_model.csv".format(doa))
        print("\n")  

train()
print("Training Complete")

Training Starts !!!!!!!
Epoch [1/50], Training Loss: 4.6227, Training Accuracy: 2.2077
SNR [0d], Validation Loss: 4.5334, Validation Accuracy: 2.8600
SNR [10d], Validation Loss: 4.0746, Validation Accuracy: 4.4171
SNR [20d], Validation Loss: 3.9653, Validation Accuracy: 4.9410
SNR [30d], Validation Loss: 3.9504, Validation Accuracy: 4.9590
SNR [40d], Validation Loss: 3.9599, Validation Accuracy: 5.0676


Epoch [2/50], Training Loss: 3.4825, Training Accuracy: 8.3235
SNR [0d], Validation Loss: 3.7453, Validation Accuracy: 6.7933
SNR [10d], Validation Loss: 3.1189, Validation Accuracy: 11.2124
SNR [20d], Validation Loss: 3.0068, Validation Accuracy: 12.5771
SNR [30d], Validation Loss: 3.0088, Validation Accuracy: 12.8752
SNR [40d], Validation Loss: 3.0267, Validation Accuracy: 12.6048


Epoch [3/50], Training Loss: 2.8921, Training Accuracy: 14.0934
SNR [0d], Validation Loss: 3.5278, Validation Accuracy: 8.6267
SNR [10d], Validation Loss: 2.8675, Validation Accuracy: 13.9943
SNR [20d], V

In [ ]:
l=0
for features, labels in test_dataset_list[1]:
  features, labels = Variable(features.cuda()), Variable(labels.cuda())
  l+=labels.size(0)

NameError: ignored

In [ ]:
l

0

In [ ]:
torch.save( model, "/content/drive/MyDrive/DOA/doa_weights/DOA_{}_model.pth".format(7))

In [ ]:
classification_model.eval()
total_valdation_loss = 0
for val_data in range(0, len(test_dataset_list)):
            
    validation_loss = 0
    validation_acc = 0
    validation_mae = 0.0
    val_correct = 0
    val_totals = 0

    with torch.no_grad():
        for features, labels in test_dataset_list[val_data]:
            features, labels = Variable(features.cuda()), Variable(labels.cuda())
            enn = classification_model(features.float())
            auto_outputs = torch.transpose(enn, 2, 3)
            auto_outputs = torch.reshape(auto_outputs, (auto_outputs.shape[0], 181, 7))
            loss = criterion(auto_outputs.cuda(), labels.type(torch.LongTensor).cuda())

            _, pred = torch.max(auto_outputs, 1)
            val_totals+= labels.reshape(-1).size(0)
            val_correct+=(pred.reshape(-1).cuda() == labels.reshape(-1)).sum().item()
            validation_loss += loss.item()
            validation_mae += torch.abs(pred.reshape(-1).cuda() - labels.reshape(-1)).sum().item()

        print(pred.reshape(-1).size(0), labels.reshape(-1).size(0))
        loss_res[10*val_data].append(validation_loss/len(test_dataset_list[val_data]))
        acc_res[10*val_data].append((100*(val_correct/val_totals)))
        print(val_correct, val_totals)
        print('SNR [{}dB], Validation Loss: {:.4f}, Validation Accuracy: {:.4f}'
              .format(val_data*10, validation_loss/len(test_dataset_list[val_data]), (100*(val_correct/val_totals))))

168 168
620 105000
SNR [0dB], Validation Loss: 5.2009, Validation Accuracy: 0.5905
168 168
594 105000
SNR [10dB], Validation Loss: 5.2010, Validation Accuracy: 0.5657
168 168
579 105000
SNR [20dB], Validation Loss: 5.2012, Validation Accuracy: 0.5514
168 168
597 105000
SNR [30dB], Validation Loss: 5.2012, Validation Accuracy: 0.5686
168 168
576 105000
SNR [40dB], Validation Loss: 5.2009, Validation Accuracy: 0.5486


In [ ]:
len(test_dataset_list[1])*64

15040